# Decision Rules

We provide a "model" to build *decision rules*, which are a series of "if-then" rules. These can be used to encode stratified values from the empirical literature. Often, in the literature, heterogenous parameters are specified for different subsets in the data. For example: if age is less than 18, then the mortality is 10%, if the age is between 18 and 30, then the mortality is 20%, and so on. This sort of series of logical implications is also MILP-representable, so we can employ our methods for it. The `DecisionRules` class helps to build such models and embed them into a Markov process.



## Setup

In [1]:
# Add the package to your Python path
import sys
sys.path.append("../")

from markovml.utils.models_ext import DecisionRules
import numpy as np

## Building a Decision Rules Model

A `DecisionRules` model is built by specifying, in natural language, the rules. In order to parse the rules, we first need to specify the features that the rules will be based on.

In [2]:
features = ['age', 'income']
model = DecisionRules(features)

And now we can specify the rules and "fit" the model.

In [3]:
rules = [
    "if age > 20 then 2.5",
    "if income >= 50000 and age < 30 then -1.0",
    "else 0.0"
]
model.fit(rules=rules)

And now we can "predict" the model by passing in a `np.array` of the features.

In [4]:
model.predict(np.array([[25,100000], [19,30000]]))

array([2.5, 0. ])

As expected, the first row yields 2.5 because the age is greater than 20, and the second row yields 0.0 because none of the rules are satisfied so it falls back to the "else" case.

You can also pass a list of dictionaries of values to the `predict` method.

In [5]:
model.predict([{'age': 25, 'income': 100000}, {'age': 19, 'income': 30000}])

array([2.5, 0. ])

There must always be an "else" case when fitting the model. As well, the value that is returned is the *first* rule that is satisfied. Here's an example where that is important.

In [6]:
features = ['age']
model = DecisionRules(features)
rules = [
    "if age > 65 then 3.0",
    "if age > 25 then 2.0",
    "if age > 12 then 1.5",
    "else 1.0"
]
model.fit(rules=rules)


The above rules can be interpreted as age-wise prices. Clearly, anyone that meets the first rule also meets the second rule, and so on. However, we always return the first satisfied rule.

In [7]:
model.predict([{'age': 70}, {'age': 30}, {'age': 20}, {'age': 10}])


array([3. , 2. , 1.5, 1. ])

The syntax for the rules is always: "if {feature} {op} {value} and {feature} {op} {value} ... then {value}". The allowed operators are: `>`, `>=`, `<`, `<=`, and `==`. The "else" case is always the last case. The reason why specifying the feature names is important is to parse the rules properly, as well as to encode the rules into the MILP formulation later on.

## Integrating into a Markov model

We have written the MILP formulation for the `DecisionRules` model. It satisfies all the same properties as when using it for predictions; for example, it is always the value of the first satisfied rule that is returned. In order to encode this series of rules into a MILP, we formulate a series of logical implications.

Here is an example of integrating the `DecisionRules` model into a Markov process.

In [14]:
from markovml.markovml import MarkovHitting
from gurobipy import GRB
markov = MarkovHitting(n_states=3, n_features=2, n_transient=2, pi=[0.7, 0.3])

# decision rules model for probabilities
dr = DecisionRules(['age', 'sex'])
dr.fit(rules = [
    "if sex == 1 and age > 20 then 0.1",
    "if sex == 0 and age > 20 then 0.05",
    "if sex == 1 and age <= 20 then 0.2",
    "if sex == 0 and age <= 20 then 0.15",
    "else 0.0"
])

markov.add_ml_model(dr)

markov.set_Q([[0.8 - markov.ml_outputs[0][0], markov.ml_outputs[0][0]],
              [0, 0.9]])

# fix features to be men from 15 to 65
markov.add_feature_constraint(markov.features[0] >= 15) # note age is feature[0]
markov.add_feature_constraint(markov.features[0] <= 65)
markov.features[1].vType = GRB.BINARY # note feature[1] is sex bc of order
markov.add_feature_constraint(markov.features[1] == 1)

markov.optimize()

{'status': 'optimal',
 'objective': 8.25,
 'values': {'pi': [0.7, 0.3],
  'Q': [[0.6, 0.2], [-0.0, 0.9]],
  'v': [7.5, 10.0],
  'features': [20.0, 1.0],
  'ml_outputs': [[0.2]]}}

So, over the specified feature space, we can expect at most 8.25 steps on average to hit the absorbing state (say, death).